In [15]:
!pip install selenium


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [16]:
!pip3 install webdriver_manager


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [17]:
# selenium: 웹 브라우저를 이용, 웹 사이트 제어를 위해 사용
# bs4: html에서 데이터 추출, html 및 xml 문서를 parsing 하기 위해 사용

### 네이버 지도 카페 상세 정보

In [18]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import os
import pandas as pd
import re

In [19]:
os.getcwd()# CSV 파일 경로 리스트
csv_file_paths = [
    # '../data/cafelist_망원동.csv',
    # '../data/cafelist_성수동.csv',
    # '../data/cafelist_연남동.csv',
    # '../data/cafelist/cafelist_한남동.csv',
    '../data/cafelist/cafelist_망원동.csv',
    '../data/cafelist/cafelist_연희동.csv',
]

sleep(3)

In [20]:
# 결과를 저장할 리스트
data_list = []

In [21]:
# 필요한 예외를 import
from selenium.common.exceptions import TimeoutException
for csv_file_path in csv_file_paths:
    data_list = []
    # CSV 파일 읽기
    df_names = pd.read_csv(csv_file_path)

    # 네이버 지도에서 크롤링
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("/Users/yangchaeyeon/Downloads/chromedriver-mac-arm64")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get("https://map.naver.com/p")

    for _, row in df_names.iterrows():
        key_word = row['name']
        print(key_word)
        
        # 네이버 지도 페이지 로딩
        driver.get("https://map.naver.com/p")

        # 검색창에 카페 이름 입력
        search = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.input_box > input.input_search')))
        search.clear()
        search.send_keys(key_word)
        search.send_keys(Keys.ENTER)
        
        # 해당 카페에 바로 가는지, 여러 카페 목록에서 클릭해서 들어가야하는지 예외 처리 
        iframe_present = False
        try:
            iframe = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe")))
            driver.switch_to.frame(iframe)
            iframe_present = True
        except TimeoutException as e:
            pass

        if not iframe_present:
            
            try:
                iframe = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchIframe"]')))
                driver.switch_to.frame(iframe)
                element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/div[2]/a[1]/div/div')))
                # 원하는 요소가 있는 경우에만 처리
                element.click()
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="entryIframe"]')))
                driver.switch_to.frame(iframe)
            except TimeoutException as e:
                # 원하는 요소를 찾지 못한 경우 다음 카페로 넘어가기
                print("해당 카페에 대한 정보를 수집할 수 없습니다.")
                continue

        # 카페 고유 ID
        try:
            ID_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'span.PXMot > a')))
            href = ID_element.get_attribute('href')
            cafe_ID = re.search(r'\d+', href).group()
            print("카페 고유 ID", cafe_ID)
        except Exception as e:
            cafe_ID = 'N/A'
        
    
        # 카페 방문자 수      
        try:
            cafe_visit = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'span.PXMot > a'))).text
            visit_count = re.sub(r'\D', '', cafe_visit)
        except Exception as e:
            cafe_visit = 'N/A'

        # 카페 블로그리뷰수
        try:
            cafe_blog = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'span.PXMot:last-child > a'))).text
            blog_count = re.sub(r'\D', '', cafe_blog)
        except Exception as e:
            cafe_blog = 'N/A'
        
        # 카페 주소
        try:
            address = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'a.PkgBl > span.LDgIH'))).text
        except Exception as e:
            address = 'N/A'



        data_list.append({
            '카페ID': cafe_ID,
            '카페이름': key_word,
            '방문자리뷰': visit_count,
            '블로그리뷰': blog_count,
            '주소': address
        })
        print(data_list[-1])
        
        sleep(7)

    # 크롬 드라이버 종료
    driver.quit()
    
    # DataFrame 생성
    df_cafe_info = pd.DataFrame(data_list)

    # 결과를 CSV로 저장
    csv_output_path = f'output_{os.path.basename(csv_file_path)}'
    df_cafe_info.to_csv(csv_output_path, index=False)

어글리베이커리
카페 고유 ID 1348768821
{'카페ID': '1348768821', '카페이름': '어글리베이커리', '방문자리뷰': '9222', '블로그리뷰': '3143', '주소': '서울 마포구 월드컵로13길 73 1층 어글리 베이커리'}
더바이글
카페 고유 ID 1416238798
{'카페ID': '1416238798', '카페이름': '더바이글', '방문자리뷰': '157', '블로그리뷰': '38', '주소': '서울 마포구 포은로 117 1층 103호'}
소설원 망원
카페 고유 ID 1988455172
{'카페ID': '1988455172', '카페이름': '소설원 망원', '방문자리뷰': '1295', '블로그리뷰': '2870', '주소': '서울 마포구 월드컵로13길 22-3'}
투떰즈업
해당 카페에 대한 정보를 수집할 수 없습니다.
비캔드
해당 카페에 대한 정보를 수집할 수 없습니다.
슈크렌느
카페 고유 ID 1256673550
{'카페ID': '1256673550', '카페이름': '슈크렌느', '방문자리뷰': '107', '블로그리뷰': '39', '주소': '서울 마포구 월드컵로11길 80 1층'}
베이크 샵 하벳
카페 고유 ID 1367299797
{'카페ID': '1367299797', '카페이름': '베이크 샵 하벳', '방문자리뷰': '21', '블로그리뷰': '21', '주소': '서울 마포구 월드컵로17길 64 1층'}
라뚜셩트
해당 카페에 대한 정보를 수집할 수 없습니다.
비전스트롤 망원점
카페 고유 ID 1672249532
{'카페ID': '1672249532', '카페이름': '비전스트롤 망원점', '방문자리뷰': '270', '블로그리뷰': '314', '주소': '서울 마포구 망원로 61 1층'}
구황작물
해당 카페에 대한 정보를 수집할 수 없습니다.
브릭베이글
카페 고유 ID 1759690653
{'카페ID': '1759690653', '카페이름': '브릭베이글', '방문자리뷰': '636', '블로그리

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=119.0.6045.159)
Stacktrace:
0   chromedriver                        0x00000001009a6004 chromedriver + 4169732
1   chromedriver                        0x000000010099dff8 chromedriver + 4136952
2   chromedriver                        0x00000001005f3500 chromedriver + 292096
3   chromedriver                        0x00000001005dba88 chromedriver + 195208
4   chromedriver                        0x00000001005db97c chromedriver + 194940
5   chromedriver                        0x00000001005db1b0 chromedriver + 192944
6   chromedriver                        0x00000001005fe724 chromedriver + 337700
7   chromedriver                        0x0000000100673870 chromedriver + 817264
8   chromedriver                        0x000000010062c5e8 chromedriver + 525800
9   chromedriver                        0x0000000100